## Happiness Model: Finding correlated features and predicting happiness

In [ ]:
### Harrison Durbin
### CE 263 - Scalable Spatial Analytics
### Final Project


#### Importing modules

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn import cross_validation
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt 
from mpl_toolkits.basemap import Basemap
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('harry.durbin', 'q3rzsjbxdu')

### Importing raw datasets

In [ ]:
# data set from World Value Survey
fn = 'WV6.csv' # approximately 85,000 individual survey results take in 60 
# countries
raw0 = np.genfromtxt(fn, delimiter=',', skiprows=1)

In [ ]:
# data set for country codes and coordinates
fn2 = 'Country_List_ISO_3166_Codes_Latitude_Longitude.csv'

name_i = [] # country name index
abbr_i = []  # country abbreviation
code_i = [] # country code
lat_i = []  # approximate latitude of country center
lng_i = []  # approximate longitude of country center

with open(fn2) as f: 
    for line in f: # go over country data, line by line
        x = line.split(',') # get a list of attributes, as strings
        name_i.append(x[0]) # country name index
        abbr_i.append(x[2]) # country abbreviation
        code_i.append(int(x[3])) # country code
        lat_i.append(float(x[4])) # approximate latitude of country center
        lng_i.append(float(x[5])) # approximate longitiude of country center

In [ ]:
lat = [] # latitudes for all points in complete raw survey file
lng = [] # longitudes for all points in complete raw survey file
abbr = [] # abbreviations for all points in complete raw survey file
name = [] # country names for all points in complete raw survey file

for y in range(len(raw0)):
    for z in range(len(name_i)):
        if int(raw0[y,1]) == int(code_i[z]):
            lat.append(lat_i[z]) 
            lng.append(lng_i[z])
            abbr.append(abbr_i[z])
            name.append(name_i[z]) 

lat = np.asarray(lat)
lng = np.asarray(lng)

In [ ]:
len(np.unique(name)) # gives number of countries in raw data

In [ ]:
X = np.vstack((lat,lng)).T # array of country coordinates for all of the raw
# data

### Creating regional country clusters based on geography

In [ ]:
# initializing minibatch kmeans
n = 10
mbkm = MiniBatchKMeans(n_clusters=n, init='k-means++', max_iter=100, batch_size=100, 
                 verbose=0, compute_labels=True, random_state=None, tol=0.0, 
                       max_no_improvement=10, init_size=None, n_init=3, 
                       reassignment_ratio=0.01)

In [ ]:
# running minibatch kmeans
mbkm.fit(X) 
mbkm_labels = mbkm.labels_ # this is an array that indicates cluster
mbkm_labels = np.asarray(mbkm_labels)
mbkm_cluster_centers = mbkm.cluster_centers_ 
mbkm_labels_unique = np.unique(mbkm_labels)  

In [ ]:
#compiling countries into 10 cluster groups
group_a = []
group_b = []
group_c = []
group_d = []
group_e = []
group_f = []
group_g = []
group_h = []
group_i = []
group_j = []

for x in range(len(name)):
    for y in range(len(mbkm_labels_unique)):
        if mbkm_labels[x] == 0:
            group_a.append(str(name[x]))
        if mbkm_labels[x] == 1:
            group_b.append(str(name[x]))
        if mbkm_labels[x] == 2:
            group_c.append(str(name[x]))
        if mbkm_labels[x] == 3:
            group_d.append(str(name[x]))
        if mbkm_labels[x] == 4:
            group_e.append(str(name[x]))
        if mbkm_labels[x] == 5:
            group_f.append(str(name[x]))
        if mbkm_labels[x] == 6:
            group_g.append(str(name[x]))
        if mbkm_labels[x] == 7:
            group_h.append(str(name[x]))      
        if mbkm_labels[x] == 8:
            group_i.append(str(name[x]))      
        if mbkm_labels[x] == 9:
            group_j.append(str(name[x]))      

In [ ]:
# print which countries are in each cluster
group_a = np.unique(group_a)
group_b = np.unique(group_b)
group_c = np.unique(group_c)
group_d = np.unique(group_d)
group_e = np.unique(group_e)
group_f = np.unique(group_f)
group_g = np.unique(group_g)
group_h = np.unique(group_h)
group_i = np.unique(group_i)
group_j = np.unique(group_j)

print 'Group A is:'
print group_a
print 'Group B is:'
print group_b
print 'Group C is:'
print group_c
print 'Group D is:'
print group_d
print 'Group E is:'
print group_e
print 'Group F is:'
print group_f
print 'Group G is:'
print group_g
print 'Group H is:'
print group_h
print 'Group I is:'
print group_i
print 'Group J is:'
print group_j

In [ ]:
raw0.shape # gives the length of raw data

In [ ]:
# create array of latitude, longitude, and cluster group
raw1 = np.vstack((lat.T,lng.T,mbkm_labels.T)) 

In [ ]:
raw1.shape # verifying correct dimensions

In [ ]:
# combines the raw data with the new lat, lng, cluster array
raw = np.vstack((raw0.T,raw1)).T 

In [ ]:
raw.shape # verifying correct dimensions

### Cleaning the data (removing rows that have any missing or inaccurate values)

In [ ]:
# delete row from array if values are missing/inaccurate in the survey
# column numbers are referenced from features list below

# V10: would you say you are 1 V. Happy, 2 Rather Happy, 
# 3 Not v. happy, 4 not at all happy
raw = raw[~(raw[:,10]<1)]

# V11: how would you describe your state of health these days? 
# 1 V. Good, 2 Good, 3 Fair, 4 Poor
raw = raw[~(raw[:,11]<1)]

# V57: Are you: 1 Married, 2 Living together,3 Divorced, 
# 4 Separated, 5 Widowed, 6 Single
raw = raw[~(raw[:,59]<1)] 

# V58: Have you had any children? 0 - 8
raw = raw[~(raw[:,60]<0)] 

# V143: do you think about the meaning and purpose of life? 
# 1 Often, 2 sometimes, 3 rarely, 4 never
raw = raw[~(raw[:,163]<1)]

# V146: how often do you pray? 1 sev time per day, 2 once/day 
# 3 sev times per week, 4 only services, 5 holy days, 6 1/yr, 7 less, 8 never
raw = raw[~(raw[:,166]<1)]

# V147: would you say you are: 1 religious, 2 not religious, 3 athiest
raw = raw[~(raw[:,167]<1)] 

# V229: are you employed now? hours/wk? YES: 1 full time, 2 partime, 3 self, 
# NO: 4 retired, 5 housewife, 6 student, 7 unempl., 8 other
raw = raw[~(raw[:,297]<1)]

# V231: are work tasks manual or intellectual? 1=mostly manual, 10=mostly intellectual
raw = raw[~(raw[:,299]<1)]

# V232: are work tasks routine or creative? 1=mostly routine, 10=mostly creative
raw = raw[~(raw[:,300]<1)]

# V238: would you describe yourself in: 1 upper class, 2 upper mid class, 
# 3 low mid, 4 working, 5 lower
raw = raw[~(raw[:,306]<1)] 

# V240: gender, 1=male, 2=female
raw = raw[~(raw[:,308]<1)]

# V242: how old are you? 00-99
raw = raw[~(raw[:,310]<1)] 

# V248: highest educational level attained? 
# 1-no formal education... 9-university level w/ degree
raw = raw[~(raw[:,318]<1)]

# V253: Size of town: 1-under 2,000 ... 8-500,000 and more
raw = raw[~(raw[:,324]<1)] 

In [ ]:
raw.shape # dimensions of the new raw data array

### Exploratory Data Analysis (EDA)

In [ ]:
happy1 = raw[:,10] # list of happiness level for each person
country1 = raw[:,1] # country code for each person
country1_unique = np.unique(country1) # gives the unique country codes

In [ ]:
country1_unique # the unique country codes

In [ ]:
# exploring the raw data

tothappiness = [] # add of the happiness levels for each country
avghappiness = [] # average the happiness levels for each country
countrycounts = [] # determine the number of surveys in each country

ones = [] # array with happiness level 1 (very happy)
twos = [] # array with happiness level 2 (rather happy)
threes = [] # array with happiness level 3 (not very happy)
fours = [] # array with happiness level 4 (not at all happy)

for x in range(len(country1_unique)):
    count = 0
    one = 0
    two = 0
    three = 0
    four = 0
    totalhappiness = 0
    for y in range(len(raw)):
        if country1[y] == country1_unique[x]:
            count += 1
            totalhappiness += happy1[y]
            if happy1[y] == 1:
                one += 1
            elif happy1[y] == 2:
                two += 1
            elif happy1[y] == 3:
                three += 1
            elif happy1[y] == 4:
                four += 1
                
    ones.append(one)
    twos.append(two)
    threes.append(three)
    fours.append(four)
    
    tothappiness.append(totalhappiness)
    countrycounts.append(count)
    avghappiness.append(totalhappiness/count)

In [ ]:
lat2 = []
lng2 = []
abbr2 = []
name2 = []

for y in range(len(country1_unique)):
    for z in range(len(name_i)):
        if int(country1_unique[y]) == int(code_i[z]):
            lat2.append(lat_i[z])
            lng2.append(lng_i[z])
            abbr2.append(abbr_i[z])
            name2.append(name_i[z]) 

In [ ]:
for row in range(len(name2)):
    print name2[row]
    print '  Average happiness level:', format(avghappiness[row],'.2f')
    print '  Total surveyed:', countrycounts[row]
    print '    Very happy percentage', ones[row]*100/countrycounts[row],'%'
    print '    Rather happy percentage', twos[row]*100/countrycounts[row],'%'
    print '    Not very happy percentage', threes[row]*100/countrycounts[row],'%'
    print '    Not happy at all percentage', fours[row]*100/countrycounts[row],'%'

In [ ]:
# plotting a bar chart of the number of surveys in each country
data = [
    go.Bar(
        x=name2,
        y= countrycounts
    )
]
plot_url = py.plot(data, filename='eda-bar')

### Splitting into Train and Test data sets

In [ ]:
# k-fold data dividing
kf = cross_validation.KFold(n=len(raw), n_folds=10, indices=None, 
                       shuffle=False, random_state=None)
for train_index, test_index in kf:
    raw_train, raw_test = raw[train_index], raw[test_index]

### Extracting features (for training set)

In [ ]:
## This is a list of the important features to extract from raw data
# the code V### after the feature name is the variable code number

# Col 10 - Happiness (1-4), V10

# Col 2 - Country Code, V2A
# Col 59 - Marital Status, V57
# Col 60 - No. of Children, V58
# Col 167 - Religious, V147
# Col 306 - Social Class, V238
# Col 307 - Scale of Income, V239
# Col 310 - Age, V242
# Col 324 - Size of Town, V253

# Col 11 - Health, V11
# Col 163 - Thoughts about life meaning, V143 ###
# Col 297 - Employment Status, V229
# Col 299 - Manual v Intellectual Work, V231 ###
# Col 300 - Routine v Creative Work, V232 ####
# Col 308 - Sex, V240
# Col 318 - Education, V248
# col 166 - Prayer, V146

In [ ]:
print raw_train.shape # dimensions of train data
print raw_test.shape # dimensions of trest data

In [ ]:
latitude = raw_train[:,430]
longitude = raw_train[:,431]
cluster = raw_train[:,432]

In [ ]:
happy = raw_train[:,10]

country = raw_train[:,1]
marital = raw_train[:,59]
kids = raw_train[:,60]
rel = raw_train[:,167]
scl = raw_train[:,306]
income = raw_train[:,307]
age = raw_train[:,310]
town = raw_train[:,324]

health = raw_train[:,11]
purpose = raw_train[:,163]
employment = raw_train[:,297]
intellect_work = raw_train[:,299]
creative_work = raw_train[:,300]
sex = raw_train[:,308]
education = raw_train[:,318]
pray = raw_train =  raw_train[:,166]

names = ['country','marital','kids','religious','social class','income','age',
         'town','health','think about life','employment','intellectual work',
         'creative work','sex','education', 'pray', 'latitude', 'longitude',
         'cluster']

### Extracting features (for test set)

In [ ]:
happy_test = raw_test[:,10]

country_test = raw_test[:,2]
marital_test = raw_test[:,59]
kids_test = raw_test[:,60]
rel_test = raw_test[:,167]
scl_test = raw_test[:,306]
income_test = raw_test[:,307]
age_test = raw_test[:,310]
town_test = raw_test[:,324]

health_test = raw_test[:,11]
purpose_test = raw_test[:,163]
employment_test = raw_test[:,297]
intellect_work_test = raw_test[:,299]
creative_work_test = raw_test[:,300]
sex_test = raw_test[:,308]
education_test = raw_test[:,318]
pray_test = raw_test[:,166]

latitude_test = raw_test[:,430]
longitude_test = raw_test[:,431]
cluster_test = raw_test[:,432]

### Compiling into a single matrix (for training set and for test set)

In [ ]:
features = np.vstack((country, marital, kids, rel, scl, income, age, town, health,
                      purpose, employment, intellect_work,creative_work,sex,education,
                      pray, latitude, longitude, cluster))
features = features.T

In [ ]:
features_test = np.vstack((country_test, marital_test, kids_test, rel_test, 
                           scl_test, income_test,age_test, town_test, health_test,
                           purpose_test, employment_test,intellect_work_test, 
                           creative_work_test, sex_test, education_test, pray_test, 
                           latitude_test, longitude_test, cluster_test))
features_test = features_test.T

In [ ]:
print features.shape
print features_test.shape

### Model Training (fitting)

In [ ]:
rf = RandomForestRegressor(random_state=0, n_estimators=100, max_depth=10)
dt = DecisionTreeClassifier(min_samples_split=20, random_state=99)
nn = KNeighborsRegressor(n_neighbors=10) 

In [ ]:
dt = dt.fit(features,happy) # decision trees

In [ ]:
nn = nn.fit(features,happy)  # nearest neighbors

In [ ]:
rf = rf.fit(features,happy) # random forest

### Predicting Happiness levels for test dataset

In [ ]:
# predictions for decision tree method
dt_predictions = dt.predict(features_test)

print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 2), dt.feature_importances_), names), 
             reverse=True)

In [ ]:
# predictions for nearest neighbor method
nn_predictions = nn.predict(features_test)

In [ ]:
# predictions for random forest method
predictions = rf.predict(features_test)

# finding feature importancea
print "Features sorted by their score:" 
p = sorted(zip(map(lambda x: round(x, 2), rf.feature_importances_), names), 
             reverse=True)
print p

In [ ]:
# putting the feature names and importance into lists
x1 = []
y1 = []
for row in range(len(p)):
    x1.append(p[row][1])
    y1.append(p[row][0])

In [ ]:
# plotting a bar chart showing feature importances
data = [
    go.Bar(
        x=x1,
        y=y1,
    )
]
plot_url = py.plot(data, filename='featureimportance-bar')

#### Determing RMSE for predictions

In [ ]:
# RMSE for nearest neighbor
nn_cv = []
for row in range(len(nn_predictions)):
    nn_cv.append(((round(nn_predictions[row]) - happy[row])**2)**0.5)
print np.mean(nn_cv)
print 100-round(np.mean(nn_cv)*100/np.mean(happy[row])), '%'

In [ ]:
# RMSE for decision tree
dt_cv = []
for row in range(len(dt_predictions)):
    dt_cv.append(((round(dt_predictions[row]) - happy[row])**2)**0.5)
print np.mean(dt_cv)

In [ ]:
# RMSE for random forest
cv = []
for row in range(len(predictions)):
    cv.append(((round(predictions[row])-happy[row])**2)**0.5)
print np.mean(cv)

In [ ]:
# nearest neighbor method has the lowest RMSE

##### Comparing individualhappiness levels in test set  predicted vs actual 

In [ ]:
from random import randrange 

In [ ]:
for i in range(20):
    r = randrange(0,len(nn_predictions),1)
    print '  pred:', int(round(nn_predictions[r])), '  act:',int(happy_test[r])

### Running predictions for the entire data set

In [ ]:
# compiling the test and train feature data
features_total = np.vstack((features_test,features))

In [ ]:
features_total.shape

In [ ]:
# predictions for the entire raw data set
happy_total_predicted = nn.predict(features_total) 

In [ ]:
tothappinessp = []
avghappinessp = []
countrycountsp = []

onesp = []
twosp = []
threesp = []
foursp = []

for x in range(len(country1_unique)):
    count = 0
    one = 0
    two = 0
    three = 0
    four = 0
    totalhappinessp = 0
    for y in range(len(raw)):
        if country1[y] == country1_unique[x]:
            count += 1
            totalhappinessp += happy_total_predicted[y]
            if round(happy_total_predicted[y]) == 1:
                one += 1
            elif round(happy_total_predicted[y]) == 2:
                two += 1
            elif round(happy_total_predicted[y]) == 3:
                three += 1
            elif round(happy_total_predicted[y]) == 4:
                four += 1

    onesp.append(one)
    twosp.append(two)
    threesp.append(three)
    foursp.append(four)
            
    tothappinessp.append(totalhappinessp)
    countrycountsp.append(count)
    avghappinessp.append(totalhappinessp/count)

In [ ]:
# for row in range(len(name2)):
#     print name2[row]
#     print '  avg happiness actual', format(avghappiness[row], '.2f')
#     print '  avg happiness predicted', format(avghappinessp[row],'.2f')
#     print '    predicted no. very happy', onesp[row]
#     print '    predicted no. rather happy', twosp[row]
#     print '    predicted no. not very happy', threesp[row]
#     print '    predicted no. not at all happy', foursp[row]

In [ ]:
# calculating the error between actual and predicted avg happiness levels
cv2=[]
for i in range(len(avghappiness)):
    cv2.append(abs(avghappiness[i]-avghappinessp[i]))
print np.mean(cv2) # the RMSE of average country happiness

In [ ]:
# sorts the final data with the happines countries on top
final_list = sorted(zip(avghappiness, avghappinessp, cv2, name2, countrycounts), 
              reverse=False)

In [ ]:
for row in range(len(final_list)):
    # list of countries (top most happy)
    print final_list[row][3] # country name
    # average country happiness (actual)
    print ' avg happiness (actl):',format(final_list[row][0],'.2f')
    # average country happiness (predicted)
    print ' avg happiness, pred:',format(final_list[row][1],'.2f') 

In [ ]:
# plot bar chart of error in country predictions
data = [
    go.Bar(
        x=name2,
        y= cv2
    )
]
plot_url = py.plot(data, filename='countryhappiness-bar')

### Creating map figure showing happiness levels across the world (actual and predicted)

In [ ]:
 # --- Build Map 
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

lons = lng2
lats = lat2
happylevels = avghappiness
happylevelsp = avghappinessp

def get_marker_color(happiness):
    # Returns different colors depending on happiness level
    if happiness < 1.75:
        return ('yo')
    elif happiness < 2.0:
        return ('mo')
    elif happiness < 2.5:
        return ('ro')
    else:
        return ('go')
    
eq_map = Basemap(projection='cyl', resolution = 'c', llcrnrlon=-150, llcrnrlat=-75,
                urcrnrlon=190, urcrnrlat=75)
    
eq_map.drawcoastlines()
eq_map.drawcountries()
eq_map.bluemarble()
eq_map.drawmapboundary()
eq_map.drawmeridians(np.arange(0, 360, 30))
eq_map.drawparallels(np.arange(-90, 90, 30))
 
min_marker_size = 14
for lon, lat, hlp in zip(lons, lats, happylevelsp):
    x,y = eq_map(lon, lat)
    msize = 1 * min_marker_size
    marker_string = get_marker_color(hlp)
    eq_map.plot(x, y, marker_string, markersize=msize)
    
    
min_marker_size = 9
for lon, lat, hl in zip(lons, lats, happylevels):
    x,y = eq_map(lon, lat)
    msize = 1 * min_marker_size
    marker_string = get_marker_color(hl)
    eq_map.plot(x, y, marker_string, markersize=msize)  
    
title_string = "Average Country Happiness\n"
plt.title(title_string)
    
plt.show() 